# Scraping Job Data from Indeed.com

First we will need to import several libraries

In [10]:
from re import A
import matplotlib as plt
import pandas as pd
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import pandas as pd

## Define a class that will store job posts on Indeed
All the relevant data

In [11]:
# this is just a utility function for printing to the console
def printhr():
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

# This class will be created from data scraped from python
# and converted to csv
class JobPost:
    def __init__(self, job_id, job_title, job_company, job_salary,
                 location, job_date_posted, job_company_link, job_page_title):
        self.job_id = job_id
        self.job_title = job_title
        self.job_company = job_company
        self.job_date_posted = job_date_posted
        self.job_company_link = job_company_link
        self.job_salary = job_salary
        self.location = location
        self.job_page_title = job_page_title

    # prints to console for debugging
    def printF(self):
        print(self.job_title)
        print(f"{self.job_company} at {self.location}")
        print(self.job_date_posted)
        print(self.job_salary)
        print(self.job_page_title)
        print(self.job_company_link)
        printhr()

    # for debugging with less text
    def desc(self):
        print(self.job_title)
        print(f"{self.job_company} at {self.location}")

    def to_dict(self):
        return {
            'job_id': self.job_id,
            'job_title': self.job_title,
            'job_company': self.job_company,
            'job_date_posted': self.job_date_posted,
            'job_company_link': self.job_company_link,
            'job_salary': self.job_salary,
            'location': self.location,
            'job_page_title': self.job_page_title
        }

# Instantiate a list
This will contain all the job posts that we collect during the scrape

In [12]:
jobs_post_list = []

### Specify some keywords that we will search for

In [13]:
keywords = ["bootcamp", "SQL", "Tableau", "degree", "data"]

# Ask user for specific inputs
This section requests specific inputs from the user and may not be necessary. running on default values for now

In [14]:
#use_default_vals = input("Use default config? (Y/N)")
use_default_vals = "Y"
while True:
  if (use_default_vals.upper() == "Y"):
    print("using default vals")
    job_query = quote("data-analyst")
    CompFilter = True
    n = 15
    deg = quote("no degree")
    url = "https://www.indeed.com/jobs?q=" + \
    job_query + "&" + deg
    from_fs = False
    print(f'Searching url {url}')
    break
  elif (use_default_vals.upper() == "N"):
    print("input your own options to the parser")
    break
  else:
    "did not recognize that. Your options are Y/N"

using default vals
Searching url https://www.indeed.com/jobs?q=data-analyst&no%20degree


In [15]:
if from_fs:
  html = open("pass1.html", "r", encoding="utf8")
else: 
  html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, "html.parser")

job_lis = soup.find("ul", "jobsearch-ResultsList")

# SEPARATE VALUES RETURNED INTO DIVS THAT CONTAIN BOTH
# .jobCard_mainContent AND .jobCardShelfContainer
result_card_list = soup.find_all("result")

# FIND THE PARENT TAGS BY CSS CLASS
results_list = soup.find_all("div", class_="cardOutline")
#print(f"Found {len(results_list)} job search results items \n")

This function wil try to pull a salary from the salary-snippet-container div

In [16]:
# CREATE A SEPARATE FUNCTION TO PARSE FOR SALARY CONTENT WITHIN THE JOB DIV
def extract_job_salary(job_div_soup) -> str:
  salary_divs = job_div_soup.find("div", "salary-snippet-container")
  salary = "Compensation Not Listed"
  if salary_divs is not None:
    salary_sub_div = salary_divs.find("div")
    for s in salary_sub_div.strings:
      if s[0] == '$':
        salary = s
  return salary

This function will pull a title from the job div

In [17]:
def extract_job_title(job_div_soup) -> str:
  title_h2 = result.find_all("h2", class_="jobTitle")
  a_tag = title_h2[0].find("a")
  title = a_tag.string
  return title

Extract the indeed's job id from a job div

In [18]:
def extract_job_id(job_soup_dev):
    title_h2 = result.find_all("h2", class_="jobTitle")
    a_tag = title_h2[0].find("a")
    job_id = a_tag["data-jk"]
    return job_id


Extract the job company from the job div

In [19]:
def extract_job_company(job_soup_div):
    company_span = result.find_all("span", class_="companyName")
    for span in company_span:
        if span.string[0] != '+':
            company = span.string
        else:
            company = span[1].string
    return company


Extract the indeed.com job url from job div

In [20]:
def extract_job_url(job_soup_div) -> str:
  company_link = result.find("a", class_="jcs-JobTitle")['href']
  company_link = 'http://indeed.com' + company_link
  return company_link

Extract the location from the job div if it is provided

In [21]:
def extract_job_location(job_soup_div) -> str:
  # EXTRACT THE LOCATION
  location_div = result.find("div", "companyLocation")
  location = location_div.string
  if location is None:
    location = "not specified"
  return location

Extract the time of the job was posted from the job div

In [22]:
def extract_job_date_created(job_soup_div) -> str:
    date_posted = "Not specified when posted"
    date_spans = result.find_all("span", class_="date")
    for d_span in date_spans:
        if (d_span.string is not None):
            date_posted = d_span.string
    return date_posted

## Iterate Through the <table> Tag that contains job

In [23]:
# ITERATE THROUGH THE <table> TAGS THAT CONTAIN JOB DATA
i = 0

print(f'Found {len(results_list)} results')
for result in results_list:
    # the for loop will be automatically be limited by the pagination on indeed
    # limited to 15 results
    if i < n:   # iterate for as many times as was specified by the config

        salary_s = extract_job_salary(result)
        title = extract_job_title(result)
        job_id = extract_job_id(result)
        company = extract_job_company(result)
        company_url = extract_job_url(result)
        location = extract_job_location(result)
        date_created = extract_job_date_created(result)
        try:
            # Create a url for the job page using indeed's url query string
            job_page_url = "https://www.indeed.com/viewjob?jk=" + job_id
            #print(f'searching at job url {job_page_url}')

            # Create a new soup of job_page_url
            job_html = urllib.request.urlopen(job_page_url).read()
            job_soup = BeautifulSoup(job_html, "html.parser")
            # Search the html soup for mentions of a specific keyword
            for keyword in keywords:
                data_mentions = job_soup.body.findAll(text=re.compile(keyword))
                print(f'{title} contains {len(data_mentions)} mentions of {keyword}')
            print(job_page_url)

            # IF THE SINGLE PAGE VIEW OF THE JOB, EXTRACT MORE DATA
            posted = job_soup.find(
                "span", class_="jobsearch-HiringInsights-entry--text").string

            # Instantiate JobPost class and append it to list
            job = JobPost(job_id, title, company, salary, location,
                          date_posted, company_link, job_soup.title)
            jobs_post_list.append(job)

        except Exception as e:
            print(e)

        # finally iterate to the next 
        i = i + 1 

Found 15 results
Remote - Data Analyst - Research and Analytics contains 0 mentions of bootcamp
Remote - Data Analyst - Research and Analytics contains 2 mentions of SQL
Remote - Data Analyst - Research and Analytics contains 0 mentions of Tableau
Remote - Data Analyst - Research and Analytics contains 2 mentions of degree
Remote - Data Analyst - Research and Analytics contains 20 mentions of data
https://www.indeed.com/viewjob?jk=a72c2452dcb67391
Data Analyst contains 0 mentions of bootcamp
Data Analyst contains 2 mentions of SQL
Data Analyst contains 2 mentions of Tableau
Data Analyst contains 2 mentions of degree
Data Analyst contains 25 mentions of data
https://www.indeed.com/viewjob?jk=d21e40d081002c62
Data Analyst contains 0 mentions of bootcamp
Data Analyst contains 4 mentions of SQL
Data Analyst contains 2 mentions of Tableau
Data Analyst contains 2 mentions of degree
Data Analyst contains 22 mentions of data
https://www.indeed.com/viewjob?jk=afae32200d9294e4
Data Analyst conta

## Can iterate through job_post_list and print to console for debugging

In [24]:

print(f"collected {len(jobs_post_list)}")
i1 = 1
for job in jobs_post_list:
    print(i1)
    job.printF()
    #job.desc()
    i1 = i1 + 1

collected 15
1
Remote - Data Analyst - Research and Analytics
Liberty Mutual Insurance at Remote
Not specified when posted
$55 - $75 an hour
<title>Remote - Data Analyst - Research and Analytics - Remote - Indeed.com</title>
http://indeed.com/company/Iprofs,-LLC/jobs/Data-Analyst-8e02dd60797712ae?fccid=4f57ff51df5aa863&vjs=3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2
Data Analyst
Purdue University at not specified
Not specified when posted
$55 - $75 an hour
<title>Data Analyst - West Lafayette, IN - Indeed.com</title>
http://indeed.com/company/Iprofs,-LLC/jobs/Data-Analyst-8e02dd60797712ae?fccid=4f57ff51df5aa863&vjs=3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
3
Data Analyst
CorTech, LLC at not specified
Not specified when posted
$55 - $75 an hour
<title>Data Analyst - Remote - Indeed.com</title>
http://indeed.com/company/Iprofs,-LLC/jobs/Data-Analyst-8e02dd60797712ae?fccid=4f57ff51df5aa863&vjs=3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
4
Data Analyst
Mainz Brady Group at Remote
No

## Append the results to a csv if they are unique
Used [this Stack Overflow post](https://stackoverflow.com/questions/34997174/how-to-convert-list-of-model-objects-to-pandas-dataframe) to figure out instantiating data frame from list of objects

In [25]:
print(len(jobs_post_list))
jobs_df = pd.DataFrame.from_records([job.to_dict() for job in jobs_post_list])
#print(jobs_df.head())
print(jobs_df.describe)
jobs_df.to_csv('data/job_posts.csv')

15
<bound method NDFrame.describe of               job_id                                       job_title  \
0   a72c2452dcb67391  Remote - Data Analyst - Research and Analytics   
1   d21e40d081002c62                                    Data Analyst   
2   afae32200d9294e4                                    Data Analyst   
3   d107ba3673a5180f                                    Data Analyst   
4   bdb6936d46f6b7d1                      Data Analyst - 100% REMOTE   
5   1e9ba23166aa6793                                Jr. Data Analyst   
6   ae7348774581dcf5                                    Data Analyst   
7   f07756912b9fc020                       Data Analyst (all levels)   
8   e97a2c4fb8e4e05f                            Product Data Analyst   
9   a51f7732a83a8476               Data Analyst (Remote Opportunity)   
10  83ed585eb98c43f4          Workplace Data Analyst - Visualization   
11  c7ddf095e634d756                      Data Analyst (100% remote)   
12  c45c1c9b3a37e2e0       